##### 第一模块（导入包模块：必须运行）

In [1]:
# #!/usr/bin/python
# -*- coding: UTF-8 -*-
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
## tk文件导入模块

##### 第二模块：(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

##### 第三模块：(程序主要功能核心：必须运行）

In [3]:
class openlaw_filter:
    #################### 
    # 此下为正则表达式筛选部分
    def __init__(self,corpus):
        """
        Param corpus:预处理的文本
        """
        self.pattern_list = [] # 初始化规则
        self.corpus = corpus # 初始化预处理文本

    def generate_patterns(self,*args):
        """
        Param *args:需要添加的规则
        return pattern_list:规则表
        """
        self.pattern_list = []
        for item in args[0]:
            pattern_ = re.compile(item)
            self.pattern_list.append(pattern_)
        return self.pattern_list
    
    def basic_mapping(self,single_document):
        """
        Param single_document:文件的单个映射
        """
        try:
            for pattern in self.pattern_list:
                if re.search(pattern,single_document):
                    return True
                else:
                    return False
        except Exception as e:
            return False
        
    def regex_filter(self,*args):
        """
        Param *args:需要添加的规则
        """
        list_args = []
        for item in args:
            list_args.append(item)
        regex_list = self.generate_patterns(list_args)
        result = list(map(self.basic_mapping,self.corpus))
        return result

    #################### 
    # 此下为余弦筛选器部分
    def _cosine_find_difference_set(self,single_para,stopwords_clean):
        """
        将stopwords和每一段文本取差集，清洗数据
        param single_para:没清洗文本集中的每一段数据
        return list_clean:每一段清洗好的文本
        """
        list_clean = set(single_para).difference(set(stopwords_clean))
        return list_clean
        
    def _cosine_data_prepare(self,corpus_clean):
        """
        把数据清理干净
        param text_list:没清洗过的文本列表
        return list_clean:清洗过的文本列表
        """
        table_segments_list = list(map(lambda x:jieba.lcut(x),corpus_clean))
        stopwords = list(map(lambda x:x.strip("\n"),codecs.open('stopwords.txt', 'r', 'utf-8').readlines()))
        stopwords_clean = list(map(lambda x:x.strip("\r"),stopwords))
        list_clean = list(map(self._cosine_find_difference_set,table_segments_list,stopwords_clean))
        return list_clean
        
    def _cosine_tfidf(self,list_clean):
        """ 
        产生tf-idf的模型
        param self.clean_text:清洗过的文本列表
        return corpus_tfidf:tf-idf模型
        """
        texts = list_clean
        # 建立词典
        dictionary = corpora.Dictionary(list_clean)
        
        # 存档词典
        dictionary.save("dict_v1.dict")
        
        # 建立词袋模型
        corpus = [dictionary.doc2bow(text) for text in texts]
        
        # 建立tf-idf模型
        tfidf = models.TfidfModel(corpus)
        
        # 将词袋模型，转换为tf-idf模型
        corpus_tfidf = tfidf[corpus]
        return corpus_tfidf
    
    def _cosine_find_similar_result(self,corpus_tfidf,article_arg,results):
        """
        使用tf-idf模型，计算文本间的余弦相似，得到结果
        param corpus_tfidf:tf-idf模型
        param target_article_arg:目标文本的下标号
        return list_final_similarities:最终的相似结果
        """
        # 创建索引
        index = similarities.MatrixSimilarity(corpus_tfidf)
        list_final_similarities = []

        # 查找最相似的十个判决结果,以第一个文本为例
        sims = index[corpus_tfidf[article_arg]]
        args = np.argsort(sims)[::-1]
        sims.sort()
        true_sims = sims[::-1]

        final_args = []
        final_args.extend(args)
        for item in list(args):
            list_final_similarities.append(results[item])
        return list_final_similarities,true_sims,args
    
    def _cosine_replace_none(self,elem):
        if elem == None:
            elem = "None"
        return elem
    
    def _cosine_similarity_filter(self,corpus_seq):
        results = list(map(self._cosine_replace_none,self.corpus.dropna()))
        list_clean = self._cosine_data_prepare(results)
        corpus_tfidf = self._cosine_tfidf(list_clean)
        result = self._cosine_find_similar_result(corpus_tfidf,corpus_seq,results)
        return result
    
    ####################
    # 其他工具:长度筛选器
    def _length_filter(self):
        results = []
        for single_corpus in self.corpus:
            try:
                if len(single_corpus) >= 100:
                    results.append(True)
                else:
                    results.append(False)
            except Exception as e:
                results.append(False)
        return results
    
    #################
    # 其他工具:追加

##### 第四模块:(函数启动模块：必须运行）

In [4]:
## 此函数为正则表达式启动函数！
def run_regex_filter(table_final,*args):
    selected_column = input("需要筛选哪一行的数据？[可填选项：庭审过程，法院意见，判决结果]")
    filter_ = openlaw_filter(table_final[selected_column])
    results = filter_.regex_filter(*args)
    return results

## 此函数为余弦筛选器启动函数!
def run_cosine_filter(table_filter,corpus_seq):
    selected_column = input("需要筛选哪一行的数据？[可填选项：庭审过程，法院意见，判决结果]")
    filter_ = openlaw_filter(table_filter[selected_column])
    results,sims,args = filter_._cosine_similarity_filter(corpus_seq)
    results = pd.Series(results)
    sims = pd.Series(sims)
    args = pd.Series(args)
    table_result = pd.DataFrame([sims,args]).T
    table_result["content"] = results
    table_result.columns = ["相似度","文本序号","判决结果内容"]
    return table_result

## 此函数为长度筛选器启动函数！
def run_length_filter(table_final):
    selected_column = input("需要筛选哪一行的数据？[可填选项：庭审过程，法院意见，判决结果]")
    filter_ = openlaw_filter(table_final[selected_column])
    results = filter_._length_filter()
    return results

## 此函数为正则表达式筛选器窗口启动函数！
def regex_filter_of_table(*args):
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_final = import_table(root)

    results = run_regex_filter(table_final,*args)
    table_final["符合要求的案件"] = results
    table_regex_filter = table_final[table_final["符合要求的案件"] == True]
    return table_regex_filter

## 此函数为余弦筛选器窗口启动函数！
def cosine_filter_of_table():
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_final = import_table(root)
    return table_final

## 此函数为长度筛选器启动函数！
def length_filter_of_table():
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_final = import_table(root)
    
    results = run_length_filter(table_final)
    table_final["符合要求的案件"] = results
    table_regex_filter = table_final[table_final["符合要求的案件"] == True]
    return table_regex_filter

#### 长度筛选器选择模块 - 选择运行！

In [5]:
import warnings
import logging
warnings.filterwarnings('ignore')

if __name__ == "__main__":
    table_length_result = length_filter_of_table()
    save_model(table_length_result)

需要筛选哪一行的数据？[可填选项：庭审过程，法院意见，判决结果]法院意见
-------------------------------------------
以下为表格的预览:
                               标题                案号 案件类型 庭审程序             案由  \
1   马文华与吴凤红生命权、健康权、身体权纠纷再审审查民事裁定书    （2016）吉民申1520号   民事   再审  生命权、健康权、身体权纠纷   
7                 林日增与林毓健身体权纠纷判决书  （2014）贵民一终字第238号   民事   二审  生命权、健康权、身体权纠纷   
8                 袁强民诉延津县公安局行政判决书    （2011）延行初字第20号   行政   一审         其他(公安)   
10            陈远海与毛明吕健康权纠纷二审民事判决书   （2018）陕09民终453号   民事   二审  生命权、健康权、身体权纠纷   
11   韩风美等与丁秋生生命权、健康权、身体权纠纷二审民事判决书  （2015）济民四终字第747号   民事   二审  生命权、健康权、身体权纠纷   

   文书类型                法院         判决日期       原告      被告  ... 法院意见2  \
1   裁定书         吉林省高级人民法院  2016年08月17日      马文华     吴凤红  ...   NaN   
7   判决书  广西壮族自治区贵港市中级人民法院  2014年11月12日      林日增     林毓健  ...   NaN   
8   判决书        河南省延津县人民法院  2011年06月25日      袁强民  延津县公安局  ...   NaN   
10  判决书      陕西省安康市中级人民法院  2018年06月26日      陈远海     毛明吕  ...   NaN   
11  判决书      山东省济南市中级人民法院  2015年11月02日  韩风美、李宗金     丁秋生  ...   NaN   

       

#### 正则表达式筛选器模块 - 选择运行！

In [6]:
## 如果要调用正则表达式筛选器，请在参数部分输入需要查找的参数
## 此部分写的话，最好
import warnings
warnings.filterwarnings('ignore')

if __name__ == "__main__":
    flag_TF = True
    list_regex = []
    while flag_TF:
        regex = input("请输入需要匹配的正则表达式:")
        list_regex.append(regex)
        flag = input("请问还要输入另一个正则表达式吗?(输入Y/N):")
        if flag == "Y":
            flag_TF = True
        else:
            flag_TF = False
    # 到底是严格匹配还是&匹配
    table_regex_result = regex_filter_of_table(*list_regex) # 在这里修改！可以在这里添加结果
    save_model(table_regex_result)

请输入需要匹配的正则表达式:正当防卫
请问还要输入另一个正则表达式吗?(输入Y/N):N
需要筛选哪一行的数据？[可填选项：庭审过程，法院意见，判决结果]法院意见
-------------------------------------------
以下为表格的预览:
                               标题                案号 案件类型 庭审程序             案由  \
1   马文华与吴凤红生命权、健康权、身体权纠纷再审审查民事裁定书    （2016）吉民申1520号   民事   再审  生命权、健康权、身体权纠纷   
7                 林日增与林毓健身体权纠纷判决书  （2014）贵民一终字第238号   民事   二审  生命权、健康权、身体权纠纷   
8                 袁强民诉延津县公安局行政判决书    （2011）延行初字第20号   行政   一审         其他(公安)   
10            陈远海与毛明吕健康权纠纷二审民事判决书   （2018）陕09民终453号   民事   二审  生命权、健康权、身体权纠纷   
11   韩风美等与丁秋生生命权、健康权、身体权纠纷二审民事判决书  （2015）济民四终字第747号   民事   二审  生命权、健康权、身体权纠纷   

   文书类型                法院         判决日期       原告      被告  ... 法院意见2  \
1   裁定书         吉林省高级人民法院  2016年08月17日      马文华     吴凤红  ...   NaN   
7   判决书  广西壮族自治区贵港市中级人民法院  2014年11月12日      林日增     林毓健  ...   NaN   
8   判决书        河南省延津县人民法院  2011年06月25日      袁强民  延津县公安局  ...   NaN   
10  判决书      陕西省安康市中级人民法院  2018年06月26日      陈远海     毛明吕  ...   NaN   
11  判决书      山东省济南市中级人民法院  2015年1

#### 余弦相似筛选器模块 - 选择运行！

In [ ]:
### 请选择使用哪个表？（通过正则处理过的表一般叫做table_filter_regex）
import warnings
import logging
warnings.filterwarnings('ignore')

if __name__ == "__main__":
    table_filter_regex = cosine_filter_of_table()
    corpus_seq_num = int(input("请选择使用表中的哪个文本作为相似初始对象？（请输入文本序号）:"))
    table = run_cosine_filter(table_filter_regex,corpus_seq_num)
    print(table)
    seq = table["文本序号"]
    table_cosine_filter = table_filter_regex.loc[seq]
    logging.warning("----------------------------------------------------------")
    save_to_excel_flag = input("是否要将其存成excel表格:(输入大写的Y或N):")
    if save_to_excel_flag == "Y":
        excel_name = input("请输入该excel表格的名字:")
        name_bash = excel_name + ".xlsx"
        table_filter_regex.to_excel(name_bash)
    elif save_to_excel_flag == "N":
        logging.warning("未有存储")
    else:
        logging.warning("输入错误")
    logging.warning("--------------------------------------------------------")
    logging.warning("下列是与第" + str(corpus_seq_num) + "份文书有着相似判决结果的" + str(len(table_cosine_filter)) + "份文书预览")
    logging.warning(table_cosine_filter.head())

请选择使用表中的哪个文本作为相似初始对象？（请输入文本序号）:1
需要筛选哪一行的数据？[可填选项：庭审过程，法院意见，判决结果]判决结果


Building prefix dict from the default dictionary ...
I1025 13:12:22.980888 4662363584 __init__.py:111] Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/gj/d82ybwn16w54j4zq9x7z_ds80000gn/T/jieba.cache
I1025 13:12:23.544315 4662363584 __init__.py:145] Dumping model to file cache /var/folders/gj/d82ybwn16w54j4zq9x7z_ds80000gn/T/jieba.cache
Loading model cost 0.618 seconds.
I1025 13:12:23.600414 4662363584 __init__.py:163] Loading model cost 0.618 seconds.
Prefix dict has been built succesfully.
I1025 13:12:23.601584 4662363584 __init__.py:164] Prefix dict has been built succesfully.
W1025 13:12:23.945359 4662363584 smart_open_lib.py:385] this function is deprecated, use smart_open.open instead
W1025 13:12:23.981972 4662363584 docsim.py:776] scanning corpus to determine the number of features (consider setting `num_features` explicitly)
W1025 13:12:24.306615 4662363584 <ipython-input-7-0c8803f71200>:13] ------------------------------------------

          相似度   文本序号                                             判决结果内容
0    1.000000    1.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项的规定，判决如下:、驳回上诉，维...
1    0.282725   98.0  ”本案到案证据无法证实被告林毓健的防卫行为过当，据此，依照《中华人民共和国民法通则》第五条、...
2    0.162257  670.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项的规定，判决如下:、驳回上诉，维...
3    0.158460  376.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上诉，维...
4    0.158460   56.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上诉，维...
5    0.158158  120.0  据此，依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上...
6    0.157992  272.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款（一）项之规定，判决如下:、驳回上诉，维持...
7    0.154662  137.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项的规定，判决如下:、驳回上诉，维...
8    0.151174  672.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上诉，维...
9    0.151174  645.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上诉，维...
10   0.151174  651.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上诉，维...
11   0.151174  725.0  依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回上诉，维...
12   0.151060  113.0  根据《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下:、驳回

In [ ]:
# 修复tkinter
# 优化代码，以及加多一些初级功能